# Import Required Libraries

We'll use [pandas](https://pandas.pydata.org/docs/index.html) for data processing and [mlxtend](http://rasbt.github.io/mlxtend/) to implement association rules mining.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Data Processing

We'll first load the data from the NHANES website ([data documentation](https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/RXQ_RX_J.htm))to a pandas dataframe. We'll only look at prescriptions taken within the last month, and we'll only need the patient identifier and drug name columns. 

In [2]:
# Load data from NHANES into prescriptions_df
prescriptions_df = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/RXQ_RX_J.XPT', encoding='UTF-8')

# Filter to prescriptions that were taken in the last month
prescriptions_df = prescriptions_df[prescriptions_df['RXDUSE'] == 1]

# Select patients (SEQN) and drug name (RXDDRUG) 
prescriptions_df = prescriptions_df[['SEQN', 'RXDDRUG']]

# View first few rows in dataframe
prescriptions_df.head()

,SEQN,RXDDRUG
2,93705.0,ENALAPRIL; HYDROCHLOROTHIAZIDE
3,93705.0,MELOXICAM
4,93705.0,OMEPRAZOLE
7,93708.0,AMLODIPINE
8,93708.0,LOSARTAN


For mlxtend to work, we'll need to label encode the drug names. This means creating a separate binary indicator column for each unique drug name. Then we'll need to group the data by the patient identifier, so that there is only one row for each unique patient identifier. As a final step, we removed all patients with no drugs.

In [3]:
# Convert data to label encoded format
prescriptions_df = pd.concat(
    [prescriptions_df['SEQN'], pd.get_dummies(prescriptions_df['RXDDRUG'])],
    axis=1
)

# Group data by patients
prescriptions_df = prescriptions_df \
    .drop(columns=['55555', '77777', '99999']) \
    .groupby('SEQN') \
    .any()

# Remove patients with no prescription drugs
prescriptions_df = prescriptions_df[prescriptions_df.sum(axis=1) > 0]

# View first few rows in dataframe
prescriptions_df.head()

,ABACAVIR; DOLUTEGRAVIR; LAMIVUDINE,ABACAVIR; LAMIVUDINE,ABATACEPT,ACARBOSE,ACEBUTOLOL,ACETAMINOPHEN; BUTALBITAL,ACETAMINOPHEN; BUTALBITAL; CAFFEINE,ACETAMINOPHEN; CODEINE,ACETAMINOPHEN; HYDROCODONE,ACETAMINOPHEN; OXYCODONE,...,VERAPAMIL,VILAZODONE,VORTIOXETINE,WARFARIN,ZAFIRLUKAST,ZALEPLON,ZIDOVUDINE,ZIPRASIDONE,ZOLPIDEM,ZONISAMIDE
SEQN,,,,,,,,,,,,,,,,,,,,,
93705.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93708.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93709.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
93713.0,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
93714.0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


# Mine Frequent Itemsets

Now we can implement association rules mining. First we'll mine the frequent itemsets using the [apriori algorithm](https://en.wikipedia.org/wiki/Apriori_algorithm). We'll generate a list of itemsets with a support of at least 0.005.

In [4]:
frequent_itemsets = apriori(prescriptions_df, min_support=0.005, use_colnames=True)
frequent_itemsets.sort_values('support', ascending=False).head()

,support,itemsets
17,0.150089,(ATORVASTATIN)
89,0.144267,(METFORMIN)
82,0.126044,(LISINOPRIL)
8,0.114148,(AMLODIPINE)
93,0.109593,(METOPROLOL)


Next let's look at association rules. From looking at the rules with the highest lift, we see that the insulin drugs are highly associated. What are other patterns that may be interesting?

In [5]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules.sort_values('lift', ascending=False, inplace=True)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
250,(INSULIN ASPART),(INSULIN GLARGINE),0.012149,0.029866,0.007593,0.625000,20.926907,0.007230,2.587024
251,(INSULIN GLARGINE),(INSULIN ASPART),0.029866,0.012149,0.007593,0.254237,20.926907,0.007230,1.324619
255,(INSULIN GLARGINE),(INSULIN LISPRO),0.029866,0.011136,0.006328,0.211864,19.024461,0.005995,1.254687
254,(INSULIN LISPRO),(INSULIN GLARGINE),0.011136,0.029866,0.006328,0.568182,19.024461,0.005995,2.246626
522,"(METOPROLOL, FUROSEMIDE)",(POTASSIUM CHLORIDE),0.017717,0.025816,0.005821,0.328571,12.727311,0.005364,1.450912
527,(POTASSIUM CHLORIDE),"(METOPROLOL, FUROSEMIDE)",0.025816,0.017717,0.005821,0.225490,12.727311,0.005364,1.268264
526,(FUROSEMIDE),"(METOPROLOL, POTASSIUM CHLORIDE)",0.053657,0.008605,0.005821,0.108491,12.607242,0.005360,1.112040
523,"(METOPROLOL, POTASSIUM CHLORIDE)",(FUROSEMIDE),0.008605,0.053657,0.005821,0.676471,12.607242,0.005360,2.925059
187,(FUROSEMIDE),(POTASSIUM CHLORIDE),0.053657,0.025816,0.014680,0.273585,10.597392,0.013295,1.341084
186,(POTASSIUM CHLORIDE),(FUROSEMIDE),0.025816,0.053657,0.014680,0.568627,10.597392,0.013295,2.193794
